# Tracking ML Experiment using MLFlow

We all have trained 1 model multiple times / say even whie tuning model - we may need to train it more than 10 times !!. If we try 10 diffeernt models. What if we can shuffle our data set we can create more differnt data sents and the number of experiments blow up. When we are wking in a team of data scientist if we want to track what happened in th nth experiment - ( what was the data / distibution/config/metrics ). Each Model run we are calling it an experimnet. Each experiment we need to capture Hyper parametrs / models / accuracy / charts/ AUC curve. This dahboard can help us track our expriments. This Dashboard help us track different experiments


Excel is a suboptimal approach. This can wer done with MLflow

Legal compliance - GDPR compliance 
1. organize all information neceaasry for experiment
2. Reproduce past runs and results using saved experiments
3. Log - Iterative improvements

In [1]:
import pandas as pd
import numpy as np

In [5]:
train_df=pd.read_csv("/workspaces/codespaces-jupyter/data/data.csv")
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP002529,Male,Yes,2,Graduate,No,6700,1750.0,230.0,300.0,1.0,Semiurban,Y
1,LP001385,Male,No,0,Graduate,No,5316,0.0,136.0,360.0,1.0,Urban,Y
2,LP001926,Male,Yes,0,Graduate,No,3704,2000.0,120.0,360.0,1.0,Rural,Y
3,LP001144,Male,Yes,0,Graduate,No,5821,0.0,144.0,360.0,1.0,Urban,Y
4,LP002562,Male,Yes,1,Not Graduate,No,5333,1131.0,186.0,360.0,NaN,Urban,Y


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            430 non-null    object 
 1   Gender             420 non-null    object 
 2   Married            427 non-null    object 
 3   Dependents         416 non-null    object 
 4   Education          430 non-null    object 
 5   Self_Employed      410 non-null    object 
 6   ApplicantIncome    430 non-null    int64  
 7   CoapplicantIncome  430 non-null    float64
 8   LoanAmount         414 non-null    float64
 9   Loan_Amount_Term   422 non-null    float64
 10  Credit_History     394 non-null    float64
 11  Property_Area      430 non-null    object 
 12  Loan_Status        430 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 43.8+ KB


## Binary Encoding of Categorical Variables

In [7]:
train_df['Gender']= train_df['Gender'].map({'Male':0, 'Female':1})
train_df['Married']= train_df['Married'].map({'No':0, 'Yes':1})
train_df['Loan_Status']= train_df['Loan_Status'].map({'N':0, 'Y':1})

In [ ]:
## Checking for Missing Values